In [54]:
import numpy as np
import pandas as pd
import random
from sklearn.cross_validation import train_test_split
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [55]:
seed = 1
file = open("test.txt", "w")
for x in range(0, 3780):
    ttemp = random.randint(70, 90)
    temp = str(ttemp)
    timpeve = random.randint(0, 1)
    impeve = str(timpeve)
    tdecibel = random.randint(60, 70) * -1
    decibel = str(tdecibel)
    halfa = random.randint(0, 23)
    halfb = random.randint(0, 59)
    if halfa == 0:
        if halfb < 10:
            time = "000" + str(halfb)
        else:
            time = "00" + str(halfb)
    else:
        if halfa < 10:
            if halfb < 10:
                time = "0" + str(halfa) + "0" + str(halfb)
            else:
                time = "0" + str(halfa) + str(halfb)

        elif halfb < 10:
            if halfa < 10:
                time = "0" + str(halfa) + "0" + str(halfb)
            else:
                time = str(halfa) + "0" + str(halfb)
        else:
            time = str(halfa) + str(halfb)
    tfail = random.randint(0, 1)
    fail = str(tfail)
    tanomaly = random.randint(0, 1)
    anomaly = str(tanomaly)
    file.write(temp + " " + impeve + " " + decibel + " " + fail + " " + time + " " + anomaly + "\n")
    random.seed()
file.close()

In [56]:
data2 = pd.read_csv('888.csv');
print(data2)

                STATION           STATION_NAME      DATE  PRCP  SNOW  TAVG  \
0     GHCND:USC00272303    EAST SANDWICH NH US  20160601  0.00   0.0 -9999   
1     GHCND:USC00272303    EAST SANDWICH NH US  20160602  0.00   0.0 -9999   
2     GHCND:USC00272303    EAST SANDWICH NH US  20160603  0.00   0.0 -9999   
3     GHCND:USC00272303    EAST SANDWICH NH US  20160604  0.31   0.0 -9999   
4     GHCND:USC00272303    EAST SANDWICH NH US  20160605  0.00   0.0 -9999   
5     GHCND:USC00272303    EAST SANDWICH NH US  20160606  2.35   0.0 -9999   
6     GHCND:USC00272303    EAST SANDWICH NH US  20160607  0.00   0.0 -9999   
7     GHCND:USC00272303    EAST SANDWICH NH US  20160608  0.04   0.0 -9999   
8     GHCND:USC00272303    EAST SANDWICH NH US  20160609  0.00   0.0 -9999   
9     GHCND:USC00272303    EAST SANDWICH NH US  20160610  0.00   0.0 -9999   
10    GHCND:USC00272303    EAST SANDWICH NH US  20160611  0.00   0.0 -9999   
11    GHCND:USC00272303    EAST SANDWICH NH US  20160612  0.05  

In [65]:
data = pd.read_csv('test.txt', sep=" ", header = None, names = ['Weather', 'Important Event?', 'Signal Level', 'Failure Nearby?', 'Time', 'Anomaly?'])
newdata = pd.concat([data, data2], axis = 1)
newdata = newdata.drop(['WT01', 'WT02','WT03','WT04','WT05','WT06','WT08','WT09', 'AWND', 'TAVG', 'STATION', 'STATION_NAME'], axis = 1)
#print(newdata)

X = newdata[['Weather', 'Important Event?', 'Signal Level', 'Failure Nearby?', 'Time', 'DATE', 'PRCP', 'SNOW', 'TMAX', 'TMIN']]
y = newdata['Anomaly?']
#print(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=0)

logreg = LogisticRegression()
logreg.fit(X_train, y_train);

y_predict = logreg.predict(X_test)

score = accuracy_score(y_test, y_predict)
print(score)

0.544973544974
